In [1]:
import os
import json
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)  # never truncate cell text
# For many columns also widen the table if needed:
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

In [3]:

!pip install kagglehub 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [kagglehub]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pavellexyr/ten-million-reddit-answers")

print("Path to dataset files:", path)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 858M/858M [00:06<00:00, 132MB/s]  

Extracting files...


Path to dataset files: /home/codespace/.cache/kagglehub/datasets/pavellexyr/ten-million-reddit-answers/versions/1


In [3]:


df_answers = pd.read_csv("/home/codespace/.cache/kagglehub/datasets/pavellexyr/ten-million-reddit-answers/versions/1/ten-million-reddit-answers.csv")

In [4]:
df_answers.head(3)

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,ge7etc6,2qh1i,askreddit,False,1606780799,https://old.reddit.com/r/AskReddit/comments/k3w3qz/someone_offers_you_100hr_to_watch_any_movie_on/ge7etc6/,"The Good, The Bad, and the Ugly, it’s just so rewatch able and it’s like 3 hours. Or Willy Wonka and the Chocolate Factory because Gene Wilder’s performance in that movie is genuine gold.",-0.3400,1
1,comment,ge7etdb,2qh1i,askreddit,False,1606780799,https://old.reddit.com/r/AskReddit/comments/k3x1kz/are_you_proud_of_your_own_nationality_why_or_why/ge7etdb/,"Well, as a Bohemian, it really just doesn't matter much anymore.",0.1931,1
2,comment,ge7et9h,2qh1i,askreddit,False,1606780798,https://old.reddit.com/r/AskReddit/comments/k48pqd/if_you_can_choose_one_power_that_has_to_do_with/ge7et9h/,Telekinesis,NaN,2


In [19]:
df_answers.loc[(df_answers["subreddit.id"]=="2qh1i")].head(3)

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,ge7etc6,2qh1i,askreddit,False,1606780799,https://old.reddit.com/r/AskReddit/comments/k3w3qz/someone_offers_you_100hr_to_watch_any_movie_on/ge7etc6/,"The Good, The Bad, and the Ugly, it’s just so rewatch able and it’s like 3 hours. Or Willy Wonka and the Chocolate Factory because Gene Wilder’s performance in that movie is genuine gold.",-0.3400,1
1,comment,ge7etdb,2qh1i,askreddit,False,1606780799,https://old.reddit.com/r/AskReddit/comments/k3x1kz/are_you_proud_of_your_own_nationality_why_or_why/ge7etdb/,"Well, as a Bohemian, it really just doesn't matter much anymore.",0.1931,1
2,comment,ge7et9h,2qh1i,askreddit,False,1606780798,https://old.reddit.com/r/AskReddit/comments/k48pqd/if_you_can_choose_one_power_that_has_to_do_with/ge7et9h/,Telekinesis,NaN,2


In [12]:
df_answers.id.nunique()

10000000

In [13]:
df_answers.shape

(10000000, 10)

In [15]:
df_answers.permalink.nunique()

10000000

In [4]:
df_questions = pd.read_csv("/home/codespace/.cache/kagglehub/datasets/pavellexyr/ten-million-reddit-answers/versions/1/ten-million-reddit-answers-questions.csv")

In [14]:
df_questions.id.nunique()

631063

In [16]:
df_questions.permalink.nunique()

631063

In [6]:
df_questions.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,hp5581,2qh1i,askreddit,False,1594447809,https://old.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,NaN,What is something that you will never forget ?,6
1,post,jkh2hg,2qh1i,askreddit,False,1604000428,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,NaN,What one moment made someone nose dive their entire career?,2
2,post,jnk1pq,2qh1i,askreddit,False,1604441310,https://old.reddit.com/r/AskReddit/comments/jnk1pq/what_is_the_absolute_greatest_pinnacle_best_song/,self.askreddit,NaN,[deleted],"What is the absolute greatest, pinnacle, best song from a musical?",2
3,post,jepd7w,2qh1i,askreddit,False,1603200144,https://old.reddit.com/r/AskReddit/comments/jepd7w/what_is_some_horrible_advice_that_you_have_been/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jepd7w/what_is_some_horrible_advice_that_you_have_been/,NaN,What is some horrible advice that you have been told?,7
4,post,jw2fs9,2qh1i,askreddit,False,1605649984,https://old.reddit.com/r/AskReddit/comments/jw2fs9/what_item_you_wanted_so_bad_when_you_were_a_kid/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jw2fs9/what_item_you_wanted_so_bad_when_you_were_a_kid/,NaN,What item you wanted so bad when you were a kid but couldn’t afford it?,2


In [5]:
import re
from typing import Optional

# Add (or reuse) cached submission_id columns for efficiency
if '_submission_id' not in df_answers.columns:
    df_answers['_submission_id'] = df_answers['permalink'].str.extract(r'/comments/([a-z0-9]+)/')
if '_submission_id' not in df_questions.columns:
    df_questions['_submission_id'] = df_questions['permalink'].str.extract(r'/comments/([a-z0-9]+)/')

# Also normalize parent ids for hierarchical traversal if needed
if '_parent_norm' not in df_answers.columns and 'parent_id' in df_answers.columns:
    df_answers['_parent_norm'] = df_answers['parent_id'].astype(str).str.replace(r'^(t1_|t3_)', '', regex=True)

In [8]:
df_answers.head(3)

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score,_submission_id
0,comment,ge7etc6,2qh1i,askreddit,False,1606780799,https://old.reddit.com/r/AskReddit/comments/k3w3qz/someone_offers_you_100hr_to_watch_any_movie_on/ge7etc6/,"The Good, The Bad, and the Ugly, it’s just so rewatch able and it’s like 3 hours. Or Willy Wonka and the Chocolate Factory because Gene Wilder’s performance in that movie is genuine gold.",-0.3400,1,k3w3qz
1,comment,ge7etdb,2qh1i,askreddit,False,1606780799,https://old.reddit.com/r/AskReddit/comments/k3x1kz/are_you_proud_of_your_own_nationality_why_or_why/ge7etdb/,"Well, as a Bohemian, it really just doesn't matter much anymore.",0.1931,1,k3x1kz
2,comment,ge7et9h,2qh1i,askreddit,False,1606780798,https://old.reddit.com/r/AskReddit/comments/k48pqd/if_you_can_choose_one_power_that_has_to_do_with/ge7et9h/,Telekinesis,NaN,2,k48pqd


In [9]:
df_questions.head(3)

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score,_submission_id
0,post,hp5581,2qh1i,askreddit,False,1594447809,https://old.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,NaN,What is something that you will never forget ?,6,hp5581
1,post,jkh2hg,2qh1i,askreddit,False,1604000428,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,NaN,What one moment made someone nose dive their entire career?,2,jkh2hg
2,post,jnk1pq,2qh1i,askreddit,False,1604441310,https://old.reddit.com/r/AskReddit/comments/jnk1pq/what_is_the_absolute_greatest_pinnacle_best_song/,self.askreddit,NaN,[deleted],"What is the absolute greatest, pinnacle, best song from a musical?",2,jnk1pq


In [13]:
# Example: Find all answers whose permalink starts with a question permalink
question_permalink = "https://old.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/"

# Filter answers whose permalink starts with the question permalink
matching_answers = df_answers[df_answers['permalink'].str.startswith(question_permalink)]

print(f"Found {len(matching_answers)} answers matching permalink prefix.")
display(matching_answers.head(5))

Found 2 answers matching permalink prefix.


,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score,_submission_id
2115014,comment,gcvtk19,2qh1i,askreddit,False,1605827283,https://old.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/gcvtk19/,"Still haven't forgotten this post, even gave it an award this time for good measure.",0.7944,2,hp5581
8865103,comment,g8yuz6i,2qh1i,askreddit,False,1602818295,https://old.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/g8yuz6i/,Still haven't forgotten this post,0.1695,1,hp5581


In [6]:
import numpy as np

# Shuffle the DataFrame first for randomness, then group and take the first row per group
shuffled = df_answers.sample(frac=1, random_state=42)
unique_answers = shuffled.groupby('_submission_id', group_keys=False).first().reset_index()

print(f"Unique answers shape: {unique_answers.shape}")
display(unique_answers.head(5))

Unique answers shape: (631064, 11)


,_submission_id,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,1080hp,comment,g8ov8gv,2qh1i,askreddit,False,1602595777,https://old.reddit.com/r/AskReddit/comments/1080hp/reddit_what_hostname_can_i_give_my_computer_to/g8ov8gv/,"**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEFORE TAKING ACTION.**\n\nYour post has been removed as it violated [Rule 2](http://www.reddit.com/r/AskReddit/wiki/index#wiki_-rule_2-). Please check that your post doesn't contain the following issues:\n\n* **Your post cannot have first person pronouns (I, me, my, etc)**\n\n* **Your post can't be specific to your situation or involve a story. Please make sure to depersonalize your question (such as removing personal pronouns) and remove any unnecessary context before posting it again.**\n\n**Please use one of these subreddits:**\n\n* Need advice? /r/advice or /r/self or /r/relationship_advice\n\n * Can't remember something? /r/TipOfMyTongue \n\n * Looking for a particular subreddit? /r/findareddit.\nPlease note - once posted, the titles cannot be edited. If your title breaks the rules, please feel free to [make a new post](/r/askreddit/submit?selftext=true) that fits within the rules.\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AskReddit) if you have any questions or concerns.*",0.9516,1
1,119m6h,comment,gc8usue,2qh1i,askreddit,False,1605354296,https://old.reddit.com/r/AskReddit/comments/119m6h/has_anybody_else_heard_of_implementation/gc8usue/,"**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEFORE TAKING ACTION.**\n\nHi there! Your question has been removed as it looks like you're asking a question that violates r/AskReddit's [Rule 3](http://www.reddit.com/r/AskReddit/wiki/index#wiki_-rule_3-) \n\nDAE (""Does Anybody Else"") questions and questions that can be answered with ""yes"" or ""no"" are not allowed in r/AskReddit. Try rephrasing your question to be more open ended, or submitting to one of the following subreddits:\n\n- /r/polls\n- /r/DoesAnybodyElse\n- /r/DAE\n- /r/WhatIsThisThing\n\n\nOtherwise if these don't fit your needs, check out the [multireddits in our sidebar](http://www.reddit.com/r/AskReddit/about/sidebar) which cover almost every topic imaginable, thanks!\n\nIf you feel this was removed in error, please [contact the mods](&lt;http://www.reddit.com/message/compose?to=%2Fr%2FAskReddit&amp;subject=Help+Related+Post+Review+Request&amp;message=My+post+was+removed+for+being+help+related,+please+review%3a%0d%0a%0d%0ahttp%3A%2F%2Fwww.reddit.com/r/AskReddit/comments/119m6h/has_anybody_else_heard_of_implementation/&gt;). \n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AskReddit) if you have any questions or concerns.*",0.8209,1
2,11c75e,comment,gco84s8,2qh1i,askreddit,False,1605661808,https://old.reddit.com/r/AskReddit/comments/11c75e/why_is_he_hurting_me/gco84s8/,"Hi there! Your post was removed because it uses the body text box. You can only use the title field to post a question in AskReddit. Per [rule 1](/r/AskReddit/wiki/index#wiki_-rule_1-), use of the body text box is prohibited. You can resubmit your post [here](&lt;https://www.reddit.com/r/askreddit/submit?selftext=true&amp;title=Why is he hurting me?&gt;) without the textbox.\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AskReddit) if you have any questions or concerns.*",-0.3031,1
3,12j5u5,comment,gc8usth,2qh1i,askreddit,False,1605354295,https://old.reddit.com/r/AskReddit/comments/12j5u5/where_can_i_find_german_news_videos_subtitled_in/gc8usth/,"**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEFORE TAKING ACTION.**\n\nYour post has been removed as it violated [Rule 2](http://www.reddit.com/r/AskReddit/wiki/index#wiki_-rule_2-). Please check that your post doesn't contain the follo

In [8]:
unique_answers.rename(columns={'body': 'answer'}, inplace=True)
unique_answers.rename(columns={'_submission_id': 'q_id'}, inplace=True)


In [10]:
df_questions.rename(columns={'title': 'question'}, inplace=True)
df_questions.rename(columns={'_submission_id': 'q_id'}, inplace=True)

In [12]:
# Merge df_questions and unique_answers
df_merged = pd.merge(df_questions, unique_answers, on='q_id', how='inner')

In [14]:
## keep question and answer in df
df_merged = df_merged[['q_id','question', 'answer']]

In [15]:
df_merged.head(3)

,q_id,question,answer
0,hp5581,What is something that you will never forget ?,"Still haven't forgotten this post, even gave it an award this time for good measure."
1,jkh2hg,What one moment made someone nose dive their entire career?,"Keep in mind, at the time Netflix was mailing DVDs to people. That was their model in 2000. They didn't start streaming until 2007. Internet in those days for most wasn't even close to what it was in 07."
2,jnk1pq,"What is the absolute greatest, pinnacle, best song from a musical?",The Pitiful Children from Be More Chill or Pick a Side from The Lightning Thief.


In [16]:
## Sample and get 1000 question answer pair
# sample 1000 rows
df_sample = df_merged.sample(n=1000, random_state=84)


In [17]:
# Fine Tuning Format
questions, answers = df_sample['question'].tolist(), df_sample['answer'].tolist()

In [18]:
qa_openai_format = [{"messages": [{"role": "system", "content": "Marv is a factual chatbot and reddit expert who likes to answer with bullets"},{"role": "user", "content": question}, {"role": "assistant", "content": answer}]} for question, answer in zip(questions, answers)]

In [19]:
qa_openai_format[5]

{'messages': [{'role': 'system',
   'content': 'Marv is a factual chatbot and reddit expert who likes to answer with bullets'},
  {'role': 'user', 'content': 'What are your thoughts on gatekeeping?'},
  {'role': 'assistant',
   'content': "It really depends on the kind of gatekeeping.\n\nGatekeeping a child from going into a bar? That's good gatekeeping.\n\nGatekeeping a woman from working in engineering, or a man from being emotional?\nThat's bad gatekeeping."}]}

In [20]:
with open('training_data.jsonl', 'w') as f:
    for entry in qa_openai_format:
        f.write(json.dumps(entry))
        f.write('\n')
        

In [21]:
# Format error checks
from collections import defaultdict
data_path = "training_data.jsonl"

with open(data_path, 'r',encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

print(f"Loaded {len(dataset)} examples from {data_path}")
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Loaded 1000 examples from training_data.jsonl
No errors found


In [11]:
df_questions.head(3)

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,question,score,q_id
0,post,hp5581,2qh1i,askreddit,False,1594447809,https://old.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,NaN,What is something that you will never forget ?,6,hp5581
1,post,jkh2hg,2qh1i,askreddit,False,1604000428,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,NaN,What one moment made someone nose dive their entire career?,2,jkh2hg
2,post,jnk1pq,2qh1i,askreddit,False,1604441310,https://old.reddit.com/r/AskReddit/comments/jnk1pq/what_is_the_absolute_greatest_pinnacle_best_song/,self.askreddit,NaN,[deleted],"What is the absolute greatest, pinnacle, best song from a musical?",2,jnk1pq


In [22]:
df_answers._submission_id.nunique()

631064

In [20]:
df_questions.shape

(631063, 13)

In [21]:
df_questions.id.nunique()

631063

In [23]:
df_questions._submission_id.nunique()

631063

In [16]:
df_questions[df_questions['permalink'].str.startswith(question_permalink)]

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score,_submission_id
0,post,hp5581,2qh1i,askreddit,False,1594447809,https://old.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/hp5581/what_is_something_that_you_will_never_forget/,NaN,What is something that you will never forget ?,6,hp5581


In [17]:
# Example: Find all answers whose permalink starts with a question permalink
question_permalink = "https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/"

# Filter answers whose permalink starts with the question permalink
matching_answers = df_answers[df_answers['permalink'].str.startswith(question_permalink)]

print(f"Found {len(matching_answers)} answers matching permalink prefix.")
display(matching_answers.head(5))

Found 4 answers matching permalink prefix.


,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score,_submission_id
6275681,comment,gaizi47,2qh1i,askreddit,False,1604001226,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/gaizi47/,"When the CEO of Ratners (British cheap jewelry company that was a big name in the 1980s and 90s) made a speech where he jokingly denigrated a lot of his company’s products. Highlights include: “[our products are so cheap] because they’re total crap” and ""[a set of our earrings are] cheaper than an M&amp;S prawn sandwich but probably wouldn't last as long”. After the speech, the valuation of Ratners plummeted by £500 million, and the company nearly collapsed. Turns out people don’t like their jewelry reminding them of that prawn sandwich they just ate...",0.1531,1,jkh2hg
6276309,comment,gaiz255,2qh1i,askreddit,False,1604001021,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/gaiz255/,"Keep in mind, at the time Netflix was mailing DVDs to people. That was their model in 2000. They didn't start streaming until 2007. Internet in those days for most wasn't even close to what it was in 07.",0.0000,2,jkh2hg
6277461,comment,gaiy585,2qh1i,askreddit,False,1604000595,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/gaiy585/,When the CEO who ran blockbuster refused to do a deal with Netflix to stream their rentals over the internet. Then proceeded to completely deny that online streaming would be a thing of the future and fought against technology at every turn only to eventually try to start their own streaming service when it was already too late...\n\nI bet absolutely no one wants to hire that dude.,-0.5840,3,jkh2hg
6277616,comment,gaiy0k9,2qh1i,askreddit,False,1604000532,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/gaiy0k9/,Mini Ladd getting exposed.,-0.0772,1,jkh2hg


In [18]:
df_questions[df_questions['permalink'].str.startswith(question_permalink)]

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score,_submission_id
1,post,jkh2hg,2qh1i,askreddit,False,1604000428,https://old.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jkh2hg/what_one_moment_made_someone_nose_dive_their/,NaN,What one moment made someone nose dive their entire career?,2,jkh2hg


In [10]:
# Helper to extract submission id from a single permalink string
def extract_submission_id(permalink: str) -> Optional[str]:
    if not isinstance(permalink, str):
        return None
    m = re.search(r'/comments/([a-z0-9]+)/', permalink)
    return m.group(1) if m else None


def get_answers_for_question(question_permalink: str, include_descendants: bool = True, sort_by: Optional[str] = 'score'):
    """Return a DataFrame of answers for a question permalink.

    Parameters
    ----------
    question_permalink : str
        The permalink of the question/submission (no trailing comment id).
    include_descendants : bool, default True
        If True, returns ALL comments in the thread (any depth).
        If False, returns only TOP-LEVEL answers (parent is the submission).
    sort_by : str | None
        Column to sort by descending (e.g., 'score', 'ups'). If missing or None, no sorting.
    """
    sub_id = extract_submission_id(question_permalink)
    if not sub_id:
        raise ValueError(f"Could not extract submission id from permalink: {question_permalink}")

    # All comments tied to this submission id
    thread_answers = df_answers[df_answers['_submission_id'] == sub_id].copy()

    if not include_descendants:
        # Keep only comments whose parent is the submission (parent_id starts with t3_<sub>)
        thread_answers = thread_answers[thread_answers['parent_id'].astype(str) == f't3_{sub_id}']

    if sort_by and sort_by in thread_answers.columns:
        thread_answers = thread_answers.sort_values(sort_by, ascending=False)

    return thread_answers



In [11]:
# Example usage with the provided question permalink
example_question = "https://old.reddit.com/r/AskReddit/comments/k3w3qz/someone_offers_you_100hr_to_watch_any_movie_on/"
answers_all = get_answers_for_question(example_question, include_descendants=True)
answers_top_level = get_answers_for_question(example_question, include_descendants=False)

print({
    'all_answers_count': len(answers_all),
    'top_level_answers_count': len(answers_top_level)
})

# Preview top 3 top-level answers (show body & score if present)
cols_show = [c for c in ['body', 'score', 'ups', 'id', 'parent_id'] if c in answers_top_level.columns]
display(answers_top_level.head(3)[cols_show])

KeyError: 'parent_id'

In [ ]:
import re
from typing import Optional

# Detect parent column dynamically (may vary in schema)
PARENT_COL = next((c for c in ['parent_id', 'parent.id', 'parent', 'parent_fullname'] if c in df_answers.columns), None)

# Add (or reuse) cached submission_id columns for efficiency
if '_submission_id' not in df_answers.columns:
    df_answers['_submission_id'] = df_answers['permalink'].str.extract(r'/comments/([a-z0-9]+)/')
if '_submission_id' not in df_questions.columns:
    df_questions['_submission_id'] = df_questions['permalink'].str.extract(r'/comments/([a-z0-9]+)/')

# Also normalize parent ids for hierarchical traversal if parent col exists
if PARENT_COL and '_parent_norm' not in df_answers.columns:
    df_answers['_parent_norm'] = df_answers[PARENT_COL].astype(str).str.replace(r'^(t1_|t3_)', '', regex=True)

# Helper to extract submission id from a single permalink string
def extract_submission_id(permalink: str) -> Optional[str]:
    if not isinstance(permalink, str):
        return None
    m = re.search(r'/comments/([a-z0-9]+)/', permalink)
    return m.group(1) if m else None


def get_answers_for_question(question_permalink: str, include_descendants: bool = True, sort_by: Optional[str] = 'score'):
    """Return a DataFrame of answers for a question permalink.

    Parameters
    ----------
    question_permalink : str
        The permalink of the question/submission (no trailing comment id).
    include_descendants : bool, default True
        If True, returns ALL comments in the thread (any depth).
        If False, returns only TOP-LEVEL answers (parent is the submission). If the parent column
        is missing, this falls back to all answers with a warning column.
    sort_by : str | None
        Column to sort by descending (e.g., 'score', 'ups'). If missing or None, no sorting.
    """
    sub_id = extract_submission_id(question_permalink)
    if not sub_id:
        raise ValueError(f"Could not extract submission id from permalink: {question_permalink}")

    # All comments tied to this submission id
    thread_answers = df_answers[df_answers['_submission_id'] == sub_id].copy()

    if not include_descendants:
        if not PARENT_COL:
            # Cannot distinguish top-level without a parent column
            thread_answers['_top_level_detection'] = 'parent_column_missing'
        else:
            # Keep only comments whose parent is the submission (parent id equals t3_<sub_id> or raw sub_id)
            parent_target_1 = f't3_{sub_id}'
            # Some dumps may store just the raw submission id; include that as fallback
            parent_target_2 = sub_id
            thread_answers = thread_answers[thread_answers[PARENT_COL].astype(str).isin([parent_target_1, parent_target_2])]

    if sort_by and sort_by in thread_answers.columns:
        thread_answers = thread_answers.sort_values(sort_by, ascending=False)

    return thread_answers

# Quick diagnostics for user clarity
print({'detected_parent_col': PARENT_COL})

# Example usage with the provided question permalink
example_question = "https://old.reddit.com/r/AskReddit/comments/k3w3qz/someone_offers_you_100hr_to_watch_any_movie_on/"
answers_all = get_answers_for_question(example_question, include_descendants=True)
answers_top_level = get_answers_for_question(example_question, include_descendants=False)

print({
    'all_answers_count': len(answers_all),
    'top_level_answers_count': len(answers_top_level)
})

# Preview top 3 top-level answers (show body & score if present)
cols_show = [c for c in ['body', 'score', 'ups', 'id', PARENT_COL] if c and c in answers_top_level.columns]
display(answers_top_level.head(3)[cols_show])

In [ ]:
import re

# Configure sample size to keep the check lightweight
SAMPLE_N = 50000

# Key columns
ans_parent_col = 'parent_id' if 'parent_id' in df_answers.columns else None
q_id_col = 'id' if 'id' in df_questions.columns else None

assert ans_parent_col and q_id_col, f"Missing join keys. answers parent={ans_parent_col}, questions id={q_id_col}"

# Sample a manageable slice of answers
ans_base = df_answers[[ans_parent_col, 'permalink']].sample(
    n=min(SAMPLE_N, len(df_answers)), random_state=0
).copy()

# Normalize IDs (remove t1_/t3_ prefixes)
ans_base['_parent_norm'] = ans_base[ans_parent_col].astype(str).str.replace(r'^(t1_|t3_)', '', regex=True)

# Build a minimal questions slice that could match the sampled answers
# 1) filter questions whose id (normalized) appears in sampled parent ids
parent_norms = set(ans_base['_parent_norm'].dropna().unique().tolist())
q_base = df_questions[[q_id_col, 'permalink']].copy()
q_base['_qid_norm'] = q_base[q_id_col].astype(str).str.replace(r'^(t1_|t3_)', '', regex=True)
q_base = q_base[q_base['_qid_norm'].isin(parent_norms)]

# Parent-id based join (expected to match many)
pairs_pid = ans_base.merge(
    q_base,
    left_on='_parent_norm', right_on='_qid_norm', how='inner',
    suffixes=('_ans', '_q')
)

# Permalink equality join (expected to be rare)
# Limit q_perma to those appearing in sampled answers to avoid large merge sets
ans_permas = set(ans_base['permalink'].dropna().unique().tolist())
q_perma = df_questions[['permalink']].copy()
q_perma = q_perma[q_perma['permalink'].isin(ans_permas)]
pairs_link = ans_base[['permalink']].merge(q_perma, on='permalink', how='inner')

print({
    'answers_rows': len(df_answers),
    'questions_rows': len(df_questions),
    'answers_sampled': len(ans_base),
    'pairs_parentId_join_sample': len(pairs_pid),
    'pairs_permalink_join_sample': len(pairs_link),
})

# Show a small sample to illustrate permalink differences
if not pairs_pid.empty:
    preview_cols = ['permalink_ans', 'permalink_q', ans_parent_col, q_id_col]
    display(pairs_pid.head(5)[preview_cols])
else:
    print("No matches found in the sample via parent_id; increase SAMPLE_N or verify columns.")

## Can `permalink` be used to join questions and answers?

Short answer: not directly. A comment’s `permalink` points to that specific comment, while a question’s `permalink` points to the parent comment. These usually differ. The reliable key is:
- answers.parent_id (strip t1_/t3_) == questions.id (comment id)

Below, we’ll verify this on a sample and show how many pairs match via `parent_id` vs exact `permalink`. 

In [5]:
!ls /home/codespace/.cache/kagglehub/datasets/pavellexyr/ten-million-reddit-answers/versions/1

ten-million-reddit-answers-questions.csv  ten-million-reddit-answers.csv


In [18]:
from datasets import load_dataset

data = load_dataset("Amod/mental_health_counseling_conversations")

In [19]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})


In [20]:
# preprocess the data for finetuning

import json

def preprocess_data(dataset):
    data = []
    for entry in dataset['train']:
        context = entry['Context']
        response = entry['Response']
        messages = [
            {"role": "user", "content": context},
            {"role": "assistant", "content": response}
        ]
        data.append({"messages": messages})
    return data

data_n = preprocess_data(data)

In [21]:
# save the preprocessed data to a JSON file

with open('fine_tune_data.jsonl','w') as f:
    for entry in data_n:
        f.write(json.dumps(entry) + '\n')

In [22]:
from collections import defaultdict

data_path = "fine_tune_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
  dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Num examples: 3512
Found errors:
missing_content: 4


In [23]:
# Importing OpenAI 
import openai
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "API_KEY"
openai.api_key=os.getenv('OPENAI_API_KEY')

client = OpenAI()

In [25]:
# fine tuning
# sending data to OpenAI

client.files.create(
    file = open('fine_tune_data.jsonl', 'rb'),
    purpose = 'fine-tune'
)

FileObject(id='file-BHKXHF3ZFBs9j3tNw3kQLqpF', bytes=4987663, created_at=1718070827, filename='fine_tune_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [26]:
client.fine_tuning.jobs.create(
    training_file='file-BHKXHF3ZFBs9j3tNw3kQLqpF',
    model='gpt-3.5-turbo'
)

FineTuningJob(id='ftjob-BIGkTrcQWT6Jt1loGndfi4AE', created_at=1718070877, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-WrpVCZI7H4QoDgRGsecsrDn2', result_files=[], seed=367529326, status='validating_files', trained_tokens=None, training_file='file-BHKXHF3ZFBs9j3tNw3kQLqpF', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [32]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-BIGkTrcQWT6Jt1loGndfi4AE', created_at=1718070877, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=7, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-WrpVCZI7H4QoDgRGsecsrDn2', result_files=[], seed=367529326, status='running', trained_tokens=None, training_file='file-BHKXHF3ZFBs9j3tNw3kQLqpF', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

In [29]:
system_prompt = "Jessy is a mental health councelling chatbot who likes to answer with bullets."
user_context = "I am a kidnapping survivor and have PTSD. I have an autoimmune disease which keeps on growing strong day by day."

In [31]:
# Check the status of the fine-tuning job
fine_tune_job_id = 'ftjob-BIGkTrcQWT6Jt1loGndfi4AE'  # Replace with your actual job ID
fine_tune_job = openai.FineTune.retrieve(id=fine_tune_job_id)

# Print the fine-tuning job details
print(fine_tune_job)

# Get the fine-tuned model ID
if fine_tune_job['status'] == 'succeeded':
    fine_tuned_model_id = fine_tune_job['fine_tuned_model']
    print(f"Fine-tuned model ID: {fine_tuned_model_id}")
else:
    print("Fine-tuning job is not yet completed or has failed.")

APIRemovedInV1: 

You tried to access openai.FineTune, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [30]:
response = client.chat.completions.create(
    model = "ft:gpt-3.5-turbo-0125",
    messages = [
        {'role':'system', "content": system_prompt},
        {'role':'user', "content": user_context}
    ]
)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `ft:gpt-3.5-turbo-0125` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}